First downloand the imdb data set from the following url:
http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

Then place the extracted folder named "aclImdb" inside the "data" folder in the same directory level as this jupyternotebook as
data/aclImdb/...

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import h5py

/Users/i318656/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# from tf.keras.models import Sequential  # This does not work!# from  
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [3]:
tf.__version__

'1.6.0'

In [4]:
import imdb

In [5]:
imdb.data_dir = "data/IMDB/"

In [6]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

In [7]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


In [8]:
data_text = x_train_text + x_test_text

In [9]:
x_train_text[1]

'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D\'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily police detective. The movie is nearly a cross-pollination of "Rosemary\'s Baby" and "The Exorcist"--but what a combination! Based on the best-seller by Jeffrey Konvitz, "The Sentinel" is entertainingly spooky, full of shocks brought off well by director Michael Winner, who mounts a thoughtfully downbe

In [10]:
y_train[1]

1.0

In [11]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [12]:
%%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 7.16 s, sys: 55.3 ms, total: 7.22 s
Wall time: 7.23 s


In [13]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [14]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

In [15]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [16]:
x_train_text[1]

'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D\'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily police detective. The movie is nearly a cross-pollination of "Rosemary\'s Baby" and "The Exorcist"--but what a combination! Based on the best-seller by Jeffrey Konvitz, "The Sentinel" is entertainingly spooky, full of shocks brought off well by director Michael Winner, who mounts a thoughtfully downbe

In [17]:
print(x_train_tokens[1])

[1153, 182, 17, 1066, 16, 815, 1457, 18, 2602, 31, 7951, 305, 4, 7888, 1210, 14, 3, 180, 18, 672, 8306, 2196, 16, 3, 1861, 35, 6, 5, 969, 15, 40, 3138, 31, 1, 5, 603, 1, 134, 16, 7951, 23, 52, 69, 1819, 1, 1245, 207, 6, 399, 8161, 6, 1313, 14, 4989, 18, 50, 7951, 1121, 82, 3, 977, 5103, 5663, 8770, 31, 3, 1998, 1982, 20, 1, 342, 1862, 177, 62, 375, 6187, 1, 5030, 585, 3, 8697, 3542, 8230, 2, 8458, 374, 7952, 2079, 5422, 23, 3, 9719, 169, 2, 9935, 6, 78, 245, 14, 3, 571, 1361, 1, 17, 6, 800, 3, 1631, 4, 8771, 978, 2, 1, 5031, 18, 48, 3, 2174, 441, 20, 1, 116, 31, 4423, 1, 8847, 6, 3746, 363, 4, 7017, 831, 122, 69, 31, 164, 498, 2297, 35, 3, 9311, 272, 16, 2788, 307, 230, 36]


In [18]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [19]:
np.mean(num_tokens)

221.27716

In [20]:
np.max(num_tokens)

2209

In [21]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

544

In [22]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94532

In [23]:
# truncating: "pre" or "post" are parameters. Truncate the tokenized list of words
# that is longer than the maximum length defined by maxlen, by filling zeros forward
# or backward.
pad = "pre"
x_train_pad = pad_sequences(x_train_tokens,
                            maxlen=max_tokens,
                            padding=pad,
                            truncating=pad)
x_test_pad = pad_sequences(x_test_tokens,
                           maxlen=max_tokens,
                           padding=pad,
                           truncating=pad)

In [24]:
# 25000 tokenized texts (train and test), each has 544 dimentions in length.
print(x_train_pad.shape, x_test_pad.shape)

(25000, 544) (25000, 544)


In [25]:
print(x_train_tokens[1])

[1153, 182, 17, 1066, 16, 815, 1457, 18, 2602, 31, 7951, 305, 4, 7888, 1210, 14, 3, 180, 18, 672, 8306, 2196, 16, 3, 1861, 35, 6, 5, 969, 15, 40, 3138, 31, 1, 5, 603, 1, 134, 16, 7951, 23, 52, 69, 1819, 1, 1245, 207, 6, 399, 8161, 6, 1313, 14, 4989, 18, 50, 7951, 1121, 82, 3, 977, 5103, 5663, 8770, 31, 3, 1998, 1982, 20, 1, 342, 1862, 177, 62, 375, 6187, 1, 5030, 585, 3, 8697, 3542, 8230, 2, 8458, 374, 7952, 2079, 5422, 23, 3, 9719, 169, 2, 9935, 6, 78, 245, 14, 3, 571, 1361, 1, 17, 6, 800, 3, 1631, 4, 8771, 978, 2, 1, 5031, 18, 48, 3, 2174, 441, 20, 1, 116, 31, 4423, 1, 8847, 6, 3746, 363, 4, 7017, 831, 122, 69, 31, 164, 498, 2297, 35, 3, 9311, 272, 16, 2788, 307, 230, 36]


In [26]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

### Question: why are we truncating the text forward, i.e. filling zeros front?

# Tokenizer Inverse Map

In [27]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

Helper-function for converting a list of tokens back to a string of words.

In [28]:
def tokens_to_string(tokens):
    # Helper-function for converting a list of tokens back to a string of words.
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [29]:
x_train_text[1]

'Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV\'s "Flamingo Road") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina\'s pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D\'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily police detective. The movie is nearly a cross-pollination of "Rosemary\'s Baby" and "The Exorcist"--but what a combination! Based on the best-seller by Jeffrey Konvitz, "The Sentinel" is entertainingly spooky, full of shocks brought off well by director Michael Winner, who mounts a thoughtfully downbe

In [30]:
print(x_train_tokens[1])

[1153, 182, 17, 1066, 16, 815, 1457, 18, 2602, 31, 7951, 305, 4, 7888, 1210, 14, 3, 180, 18, 672, 8306, 2196, 16, 3, 1861, 35, 6, 5, 969, 15, 40, 3138, 31, 1, 5, 603, 1, 134, 16, 7951, 23, 52, 69, 1819, 1, 1245, 207, 6, 399, 8161, 6, 1313, 14, 4989, 18, 50, 7951, 1121, 82, 3, 977, 5103, 5663, 8770, 31, 3, 1998, 1982, 20, 1, 342, 1862, 177, 62, 375, 6187, 1, 5030, 585, 3, 8697, 3542, 8230, 2, 8458, 374, 7952, 2079, 5422, 23, 3, 9719, 169, 2, 9935, 6, 78, 245, 14, 3, 571, 1361, 1, 17, 6, 800, 3, 1631, 4, 8771, 978, 2, 1, 5031, 18, 48, 3, 2174, 441, 20, 1, 116, 31, 4423, 1, 8847, 6, 3746, 363, 4, 7017, 831, 122, 69, 31, 164, 498, 2297, 35, 3, 9311, 272, 16, 2788, 307, 230, 36]


We can recreate this text except for punctuation and other symbols, by converting the list of tokens back to words:

In [31]:
tokens_to_string(x_train_tokens[1])

"bizarre horror movie filled with famous faces but stolen by raines later of tv's road as a pretty but somewhat unstable model with a smile who is to pay for her attempted by the to hell the scenes with raines are very well captured the mood music is perfect deborah is charming as pal but when raines moves into a creepy brooklyn heights inhabited by a blind priest on the top floor things really start cooking the neighbors including a fantastically wicked meredith and kinky couple sylvia miles beverly are a diabolical lot and eli is great fun as a police detective the movie is nearly a cross of rosemary's baby and the exorcist but what a combination based on the best by jeffrey the sentinel is spooky full of shocks brought off well by director michael winner who a downbeat ending with skill 1 2 from"

# Create the Recurrent Neural Network

### GRU
We are going to use GRU as a cell of RNN.

![GRU Architecture and Equations](img/GRU.png)


Original Paper Source: https://arxiv.org/pdf/1406.1078v3.pdf

In [32]:
model = Sequential()

In [33]:
embedding_size = 8

In [34]:
# num_words: the vocabulary size used as an input, 10000 in our case since we chose top 10000 frequent words.
# output_dim: arbitrary dimension of embedding output layer.
# input_length: the maximum length of the input sequence, in our case 544.
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [35]:
# units: positive integer, that is the size of output dimension.
model.add(GRU(units=16, return_sequences=True))

In [36]:
model.add(GRU(units=8, return_sequences=True))

In [37]:
model.add(GRU(units=4))

In [38]:
model.add(Dense(1, activation='sigmoid'))

### Adam (Adaptive moment estimation)
Now we are going to use the optimizer called Adam, which is one of the many optimizers, definition delineated below. It was developed in 2015 by group of authers including Diederik P. Kingma. It has relatively high learning efficiency, however it does not necessary guarantee that Adam should be used for all datasets and networks.

Given parameters $w^{(t)}$ and a loss function $L^{(t)}$, where $t$ indexes the current training iteration (indexed at $0$), Adam's parameter update is given by:
\begin{equation*}
m_w ^ {(t+1)} \leftarrow \beta_1 m_w ^ {(t)} + (1 - \beta_1) \nabla _w L ^ {(t)}\\
v_w ^ {(t+1)} \leftarrow \beta_2 v_w ^ {(t)} + (1 - \beta_2) (\nabla _w L ^ {(t)})^2\\
\hat{m}_w = \frac{m_w ^ {(t+1)}}{1 - \beta_1 ^{t+1}}\\
\hat{v}_w = \frac{ v_w ^ {(t+1)}}{1 - \beta_2 ^{t+1}}\\
w ^ {(t+1)} \leftarrow w ^ {(t)} - \eta \frac{\hat{m}_w}{\sqrt{\hat{v}_w} + \epsilon}\\
\end{equation*}
where $\epsilon$  is a small scalar used to prevent division by $0$, and $\beta _1$ and $\beta _2$ are the forgetting factors for gradients and second moments of gradients, respectively. Squaring and square-rooting is done elementwise.

Recommended values for parameters in the paper:
$\eta=0.001, \beta_1=0.9, \beta_2=0.999, \epsilon=10^{-8}$
and $m_0=0, v_0=0$ for initial values.

Original Paper Source: https://arxiv.org/pdf/1412.6980.pdf

In [39]:
optimizer = Adam(lr=1e-3)

We are going to compile the defined model for it to be ready for training.

In [40]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 16)          1200      
_________________________________________________________________
gru_2 (GRU)                  (None, None, 8)           600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


# Training the defined RNN

In [42]:
%%time
model.fit(x_train_pad,
          y_train,
          validation_split=0.05, 
          epochs=3, 
          batch_size=64)

Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================]23750/23750 [==============================] - 382s 16ms/step - loss: 0.6729 - acc: 0.5744 - val_loss: 0.6416 - val_acc: 0.6384

Epoch 2/3
23750/23750 [==============================]23750/23750 [==============================] - 368s 15ms/step - loss: 0.4486 - acc: 0.7921 - val_loss: 0.0639 - val_acc: 0.9976

Epoch 3/3
23750/23750 [==============================]23750/23750 [==============================] - 377s 16ms/step - loss: 0.3133 - acc: 0.8733 - val_loss: 0.6484 - val_acc: 0.7200

CPU times: user 32min 21s, sys: 9min 20s, total: 41min 41s
Wall time: 18min 48s


model.save(filepath) enables you to save a model as HDF5 file.
HDF5 file contains the following items:
<ul>
  <li>Model structure that can be reconstracted</li>
  <li>Weights of model</li>
  <li>Learning parameter setting: loss, optimizer</li>
  <li>Status of the optimizer, so that after the end of training, it enables smooth starts of the retraining</li>
</ul>

In [43]:
# For example:
# from keras.models import load_model

# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

You can also save only the model architecture by

In [44]:
# # save as JSON
# json_string = model.to_json()

# # save as YAML
# yaml_string = model.to_yaml()

For more details refer to the keras offical documents:
https://keras.io/ja/getting-started/faq/#keras-model

In [45]:
model.save("model/model_1.h5")

# Model Performance on Test-Set

In [46]:
model = load_model("model/model_1.h5")

In [47]:
%%time
result = model.evaluate(x_test_pad, y_test)

25000/25000 [==============================]25000/25000 [==============================] - 115s 5ms/step

CPU times: user 3min 40s, sys: 1min 4s, total: 4min 45s
Wall time: 1min 55s


In [48]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 83.56%


# New Data

In [49]:
model = load_model("model/model_1.h5")

In [50]:
text0 = "This is excellent! This is by far the best movie I have ever seen in my life!"
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
text9 = "This is a complete waste of time. The worst movie ever!"
texts = [text0, text1, text2, text3, text4, text5, text6, text7, text8, text9]

In [51]:
tokens = tokenizer.texts_to_sequences(texts)

In [52]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(10, 544)

In [53]:
predicted = model.predict(tokens_pad)
print(predicted)

[[0.92068624]
 [0.8908813 ]
 [0.87077683]
 [0.48089698]
 [0.8431827 ]
 [0.36755407]
 [0.52853155]
 [0.74753517]
 [0.1889803 ]
 [0.05744076]]


The closer a value is to $0$ more negative sentiment it has and the closer it is to $1$ more positie sentiment it has.